# Analyse de la Rupture de Fusibles à Haute Vitesse

Ce notebook contient le code complet pour analyser des vidéos de radiographie à rayons X à haute vitesse de fusibles industriels. Il mesure la distance entre les éléments du fusible pendant les événements de rupture.

## Aperçu

Les fusibles à haute capacité de rupture (HRC) sont des composants de sécurité essentiels dans les systèmes électriques. Lorsqu'un court-circuit se produit, ces fusibles doivent se rompre rapidement pour protéger le circuit. Ce notebook analyse les vidéos de radiographie à rayons X des événements de rupture de fusibles pour mesurer la distance entre les éléments du fusible au fil du temps, fournissant des informations sur la dynamique de rupture.

## 1. Importation des bibliothèques

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional
import pandas as pd
import time
import logging
from dataclasses import dataclass
from scipy.signal import savgol_filter
from skimage import measure

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## 2. Module de traitement vidéo

In [ ]:
@dataclass
class VideoFrame:
    """Classe pour stocker les données et métadonnées d'une frame vidéo."""
    index: int
    data: np.ndarray
    timestamp: Optional[float] = None


class VideoProcessor:
    """Classe pour traiter les vidéos à haute vitesse d'événements de rupture de fusibles."""
    
    def __init__(self, video_path: str):
        """
        Initialise le processeur vidéo.
        
        Args:
            video_path: Chemin vers le fichier vidéo
        """
        self.video_path = video_path
        self.video_obj = None
        self.frames = []
        self.height = 0
        self.width = 0
        self.fps = 0
        self.frame_count = 0
        
    def load_video(self) -> bool:
        """
        Charge le fichier vidéo et extrait les propriétés de base.
        
        Returns:
            bool: True si la vidéo a été chargée avec succès, False sinon
        """
        try:
            self.video_obj = cv2.VideoCapture(self.video_path)
            if not self.video_obj.isOpened():
                print(f"Erreur: Impossible d'ouvrir le fichier vidéo {self.video_path}")
                return False
                
            # Obtenir les propriétés de la vidéo
            self.width = int(self.video_obj.get(cv2.CAP_PROP_FRAME_WIDTH))
            self.height = int(self.video_obj.get(cv2.CAP_PROP_FRAME_HEIGHT))
            self.fps = self.video_obj.get(cv2.CAP_PROP_FPS)
            self.frame_count = int(self.video_obj.get(cv2.CAP_PROP_FRAME_COUNT))
            
            print(f"Vidéo chargée: {self.width}x{self.height}, {self.fps} fps, {self.frame_count} frames")
            return True
            
        except Exception as e:
            print(f"Erreur lors du chargement de la vidéo: {str(e)}")
            return False
    
    def extract_all_frames(self) -> List[VideoFrame]:
        """
        Extrait toutes les frames de la vidéo et les stocke.
        
        Returns:
            List[VideoFrame]: Liste des frames vidéo
        """
        if self.video_obj is None or not self.video_obj.isOpened():
            if not self.load_video():
                return []
        
        self.frames = []
        frame_idx = 0
        
        while True:
            ret, frame = self.video_obj.read()
            if not ret:
                break
                
            # Convertir en niveaux de gris si la frame est en couleur
            if len(frame.shape) == 3:
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            else:
                gray_frame = frame
                
            # Stocker la frame
            self.frames.append(VideoFrame(
                index=frame_idx,
                data=gray_frame,
                timestamp=frame_idx / self.fps if self.fps > 0 else None
            ))
            
            frame_idx += 1
            
        print(f"Extraction de {len(self.frames)} frames")
        return self.frames
    
    def get_frame(self, index: int) -> Optional[np.ndarray]:
        """
        Récupère une frame spécifique par son index.
        
        Args:
            index: Index de la frame
            
        Returns:
            np.ndarray: Données de la frame ou None si l'index est invalide
        """
        if 0 <= index < len(self.frames):
            return self.frames[index].data
        return None
    
    def release(self):
        """Libère les ressources vidéo."""
        if self.video_obj is not None and self.video_obj.isOpened():
            self.video_obj.release()

## 3. Module de traitement d'image

In [ ]:
class FuseImageProcessor:
    """Classe pour traiter les images à rayons X de fusibles et mesurer les distances de rupture."""
    
    def __init__(self, calibration_value_mm: float = 2.0):
        """
        Initialise le processeur d'image de fusible.
        
        Args:
            calibration_value_mm: La hauteur H connue du fusible en mm (par défaut: 2.0)
        """
        self.calibration_value_mm = calibration_value_mm
        self.pixels_per_mm = None
        self.calibrated = False
    
    def preprocess_image(self, image: np.ndarray) -> np.ndarray:
        """
        Prétraite l'image pour une meilleure segmentation.
        
        Args:
            image: Image en niveaux de gris d'entrée
            
        Returns:
            np.ndarray: Image prétraitée
        """
        # Appliquer un flou gaussien pour réduire le bruit
        blurred = cv2.GaussianBlur(image, (5, 5), 0)
        
        # Utiliser le seuillage d'Otsu qui est meilleur pour les images bimodales comme les rayons X
        _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Appliquer des opérations morphologiques pour nettoyer l'image binaire
        kernel = np.ones((3, 3), np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
        
        # Appliquer une fermeture pour combler les petits trous
        kernel = np.ones((5, 5), np.uint8)
        cleaned = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)
        
        return cleaned
    
    def segment_fuse_elements(self, image: np.ndarray) -> Tuple[np.ndarray, List]:
        """
        Segmente les éléments du fusible (parties noires) dans l'image.
        
        Args:
            image: Image en niveaux de gris d'entrée
            
        Returns:
            Tuple contenant:
                - Masque binaire des éléments segmentés
                - Liste des propriétés pour chaque région détectée
        """
        # Prétraiter l'image
        binary = self.preprocess_image(image)
        
        # Trouver les composantes connexes
        labeled_img = measure.label(binary)
        regions = measure.regionprops(labeled_img)
        
        # Filtrer les régions par surface pour éliminer le bruit
        min_area = 100  # Ajuster en fonction de la résolution de l'image
        valid_regions = [r for r in regions if r.area >= min_area]
        
        # Créer un masque avec uniquement les régions valides
        mask = np.zeros_like(binary)
        for region in valid_regions:
            for coord in region.coords:
                mask[coord[0], coord[1]] = 255
                
        return mask, valid_regions
    
    def calibrate(self, reference_image: np.ndarray) -> float:
        """
        Calibre le système de mesure en utilisant une image de référence.
        
        Args:
            reference_image: Image de référence où H est connu
            
        Returns:
            float: Facteur de calibration en pixels par millimètre
        """
        # Segmenter les éléments du fusible
        mask, regions = self.segment_fuse_elements(reference_image)
        
        if len(regions) < 1:
            raise ValueError("Aucun élément de fusible détecté dans l'image de référence")
        
        # Trouver la hauteur (H) en pixels
        # Pour la calibration, nous utilisons la première frame où le fusible est intact
        heights = [r.bbox[2] - r.bbox[0] for r in regions]
        h_pixels = max(heights)  # Utiliser la plus grande hauteur comme H
        
        # Calculer les pixels par mm
        self.pixels_per_mm = h_pixels / self.calibration_value_mm
        self.calibrated = True
        
        print(f"Calibration terminée: {self.pixels_per_mm:.2f} pixels/mm")
        return self.pixels_per_mm
    
    def measure_distance(self, image: np.ndarray) -> Optional[float]:
        """
        Mesure la distance entre les éléments du fusible en millimètres.
        
        Args:
            image: Image en niveaux de gris d'entrée
            
        Returns:
            float: Distance en millimètres ou None si la mesure a échoué
        """
        if not self.calibrated:
            raise ValueError("Calibration requise avant la mesure")
        
        # Segmenter les éléments du fusible
        mask, regions = self.segment_fuse_elements(image)
        
        # Vérifier si nous avons assez de régions pour mesurer
        if len(regions) < 2:
            # Si nous n'avons qu'une seule région, le fusible est probablement intact
            if len(regions) == 1:
                # Vérifier si la région couvre la majeure partie de la largeur
                region = regions[0]
                width = image.shape[1]
                region_width = region.bbox[3] - region.bbox[1]
                
                if region_width > 0.5 * width:
                    # C'est probablement un fusible intact
                    return 0.0
            
            # Pour les frames avant le début de la rupture
            # Vérifier s'il s'agit d'une frame précoce (fusible intact)
            # Rechercher des pixels sombres au milieu de l'image
            h, w = image.shape
            center_region = image[h//4:3*h//4, w//4:3*w//4]
            if np.mean(center_region) < 100:  # Ajuster le seuil si nécessaire
                return 0.0
                
            return None
        
        # Pour les frames avec plusieurs régions, nous devons identifier les principales parties du fusible
        
        # Filtrer les régions par taille pour se concentrer sur les principales parties du fusible
        min_area_ratio = 0.01  # Surface minimale en fraction de la plus grande région
        largest_area = max(r.area for r in regions)
        significant_regions = [r for r in regions if r.area > min_area_ratio * largest_area]
        
        if len(significant_regions) < 2:
            return 0.0  # Pas assez de régions significatives
            
        # Trier les régions horizontalement (par coordonnée x)
        sorted_regions = sorted(significant_regions, key=lambda r: r.centroid[1])
        
        # Trouver les régions significatives les plus à gauche et les plus à droite
        left_regions = sorted_regions[:len(sorted_regions)//2]
        right_regions = sorted_regions[len(sorted_regions)//2:]
        
        if not left_regions or not right_regions:
            return 0.0
            
        # Trouver le point le plus à droite de toutes les régions de gauche
        left_edge = max(r.bbox[1] + r.bbox[3] for r in left_regions)  # bord droit des régions de gauche
        
        # Trouver le point le plus à gauche de toutes les régions de droite
        right_edge = min(r.bbox[1] for r in right_regions)  # bord gauche des régions de droite
        
        # Calculer la distance
        distance_pixels = max(0, right_edge - left_edge)
        distance_mm = distance_pixels / self.pixels_per_mm
        
        # Appliquer un seuil pour éviter le bruit
        if distance_mm < 0.1:  # Distance minimale significative
            return 0.0
            
        return distance_mm
    
    def visualize_measurement(self, image: np.ndarray, distance_mm: float) -> np.ndarray:
        """
        Crée une visualisation de la mesure sur l'image.
        
        Args:
            image: Image en niveaux de gris d'entrée
            distance_mm: Distance mesurée en millimètres
            
        Returns:
            np.ndarray: Image de visualisation avec annotations
        """
        # Convertir l'image en niveaux de gris en couleur pour la visualisation
        vis_img = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        
        # Segmenter les éléments du fusible
        mask, regions = self.segment_fuse_elements(image)
        
        # Dessiner les contours autour des régions détectées
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(vis_img, contours, -1, (0, 255, 0), 2)
        
        # Ajouter le texte de mesure de distance
        cv2.putText(
            vis_img, 
            f"d = {distance_mm:.3f} mm", 
            (10, 30), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.7, 
            (0, 0, 255), 
            2
        )
        
        return vis_img